In [1]:
%matplotlib inline

import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys

from PIL import Image
from utils.helpers import *

In [2]:
from src.unet_keras import *

/Users/olivier/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/olivier/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/olivier/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/olivier/anaconda3/envs/ml/lib/python3.7/site-

In [3]:
# Loaded a set of images
root_dir = "data/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(20, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

Loading 20 images
satImage_052.png
Loading 20 images
satImage_052.png


In [4]:
x = np.asarray(imgs)
y = np.expand_dims(np.asarray(gt_imgs), axis=3)

x_train = x[:12]
y_train = y[:12]

x_val = x[12:15]
y_val = y[12:15]

x_test = x[15:]
y_test = y[15:]

In [5]:
x_train.shape

(12, 400, 400, 3)

In [6]:
y_train.shape

(12, 400, 400, 1)

In [7]:
x_test[:1].shape

(1, 400, 400, 3)

In [13]:
x_test[0]*255

array([[[121., 113.,  99.],
        [117., 108.,  94.],
        [122., 113., 100.],
        ...,
        [110., 107., 100.],
        [108., 105.,  97.],
        [106., 103.,  96.]],

       [[117., 107.,  94.],
        [117., 109.,  94.],
        [126., 118., 105.],
        ...,
        [105., 102.,  94.],
        [101.,  98.,  91.],
        [100.,  97.,  90.]],

       [[117., 107.,  93.],
        [118., 108.,  94.],
        [128., 121., 107.],
        ...,
        [103.,  99.,  92.],
        [104., 100.,  93.],
        [103., 100.,  93.]],

       ...,

       [[ 63.,  63.,  57.],
        [ 84.,  79.,  69.],
        [ 91.,  83.,  71.],
        ...,
        [173., 168., 150.],
        [169., 163., 146.],
        [167., 161., 144.]],

       [[ 69.,  71.,  60.],
        [ 83.,  80.,  68.],
        [ 90.,  82.,  70.],
        ...,
        [146., 142., 130.],
        [141., 137., 125.],
        [138., 135., 122.]],

       [[ 67.,  70.,  57.],
        [ 82.,  81.,  70.],
        [ 85.,  

In [8]:
IMG_SIZE = 400
NUM_CHANNELS = 3
NUM_FILTER = 16
FILTER_SIZE = 3

BATCH_SIZE = 16
NUM_EPOCHS = 10

In [9]:
model = unet_model(IMG_SIZE, NUM_CHANNELS, NUM_FILTER, FILTER_SIZE)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
model = train_model(model, x_train, y_train, x_val, y_val, BATCH_SIZE, NUM_EPOCHS)

Train on 12 samples, validate on 3 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
12/12 [==============================] - 33s 3s/sample - loss: 14.7517 - acc: 0.1161 - val_loss: 2.9783 - val_acc: 0.0019
Epoch 2/10
12/12 [==============================] - 15s 1s/sample - loss: 14.1337 - acc: 0.0968 - val_loss: 3.0993 - val_acc: 0.0167
Epoch 3/10
12/12 [==============================] - 16s 1s/sample - loss: 13.5160 - acc: 0.0808 - val_loss: 4.3353 - val_acc: 0.0366
Epoch 4/10
12/12 [==============================] - 16s 1s/sample - loss: 12.9132 - acc: 0.0733 - val_loss: 9.4842 - val_acc: 0.0166
Epoch 5/10
12/12 [==============================] - 16s 1s/sample - loss: 12.3245 - acc: 0.0704 - val_loss: 20.7322 - val_acc: 0.0039
Epoch 6/10
12/12 [==============================] - 16s 1s/sample - loss: 11.7687 - acc: 0.0713 - val_loss: 30.0599 - val_acc: 0.0033
Epoch 7/10
12/12 [==============================] - 15s 1s/sample - loss: 11.2555 - acc: 0.0771 - val_loss: 3

In [11]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test)
print('test loss, test acc:', results)


# Evaluate on test data
5/5 [==============================] - 3s 515ms/sample - loss: 60.7680 - acc: 0.0536
test loss, test acc: [60.767974853515625, 0.053605]


In [12]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(x_test)
print('predictions shape:', predictions.shape)


# Generate predictions for 3 samples
predictions shape: (1, 400, 400, 1)
